In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from rex import NSRDBX
import matplotlib.pyplot as plt


## PySAM part

In [2]:
import PySAM.Pvsamv1 as PV
import PySAM.Grid as Grid
import PySAM.Utilityrate5 as UtilityRate
import PySAM.Cashloan as Cashloan
import pathlib
import json
import os
import pandas as pd
import PySAM
from platform import python_version

sif2 = 'AgriPV_SAMJsons'



In [3]:
print(PySAM.__version__)
print(pd.__version__)
print(python_version())

4.0.0
1.5.2
3.9.5


In [4]:
file_names = ["pvsamv1", "grid", "utilityrate5", "cashloan"]

pv2 = PV.new()  # also tried PVWattsSingleOwner
grid2 = Grid.from_existing(pv2)
ur2 = UtilityRate.from_existing(pv2)
so2 = Cashloan.from_existing(grid2, 'FlatPlatePVCommercial')



In [5]:
for count, module in enumerate([pv2, grid2, ur2, so2]):
    filetitle= 'AgriPV_SAM' + '_' + file_names[count] + ".json"
    with open(filetitle, 'r') as file:
        data = json.load(file)
        for k, v in data.items():
            if k == 'number_inputs':
                continue
            try:
                module.value(k, v)
            except AttributeError:
                # there is an error is setting the value for ppa_escalation
                print(module, k, v)

In [6]:
pv2.SolarResource.solar_resource_file = ''

In [13]:
pv2.SolarResource.replace('solar_resource_file') # Removes the file so it doesnt try to read it.

In [14]:
# Sanity check , This should give an exception:
#pv2.SolarResource.solar_resource_file

In [15]:
df_weather_gid = pd.read_pickle("../df_weather_gid.pkl")
print(df_weather_gid)
meta_dict =  {'latitude': 39.970001220703125, 'longitude': -105.05999755859375, 'elevation': 1614, 'timezone': -7, 'country': 'United States', 'state': 'Colorado', 'county': 'Boulder'}

par                        air_temperature  dhi  dni  ghi  surface_albedo  \
time_index                                                                  
2021-01-01 00:30:00-07:00            -13.8    0    0    0             0.8   
2021-01-01 01:30:00-07:00            -13.7    0    0    0             0.8   
2021-01-01 02:30:00-07:00            -13.6    0    0    0             0.8   
2021-01-01 03:30:00-07:00            -13.5    0    0    0             0.8   
2021-01-01 04:30:00-07:00            -13.5    0    0    0             0.8   
...                                    ...  ...  ...  ...             ...   
2021-12-31 19:30:00-07:00            -14.0    0    0    0             0.8   
2021-12-31 20:30:00-07:00            -13.8    0    0    0             0.8   
2021-12-31 21:30:00-07:00            -13.6    0    0    0             0.8   
2021-12-31 22:30:00-07:00            -13.6    0    0    0             0.8   
2021-12-31 23:30:00-07:00            -13.7    0    0    0             0.8   

In [16]:
pv2.SolarResource.assign({'solar_resource_data':{'dn':list(df_weather_gid.dni),
                                                   'df':list(df_weather_gid.dhi),
                                                   'tdry':list(df_weather_gid.air_temperature),
                                                   'wspd':list(df_weather_gid.wind_speed),
                                                   'lat':meta_dict['latitude'],
                                                   'lon':meta_dict['longitude'],
                                                   'tz':meta_dict['timezone'],
                                                   'Elevation':meta_dict['elevation'],
                                                   'Year':list(df_weather_gid.index.year),
                                                   'Month':list(df_weather_gid.index.month),
                                                   'Day':list(df_weather_gid.index.day),
                                                   'Hour':list(df_weather_gid.index.hour),
                                                   'Minute':list(df_weather_gid.index.minute),
                                                   'alb':list(df_weather_gid.surface_albedo)}})

In [17]:
# Making sure correct parameters are set on these 
pv2.SolarResource.irrad_mode = 0     # 0 for DNI and DHI input use
pv2.SolarResource.use_wf_albedo = 1  # 1 for using weather file albedo

In [18]:
# Assign albedo otherwise it fails even if use_wf_albedo=1, 'bug' reported
pv2.SolarResource.albedo = list(df_weather_gid.surface_albedo) # 
pv2.SolarResource.albedo = [0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2]

In [19]:
# help(pv2.SolarResource)

In [20]:
grid2.SystemOutput.gen = [0] * 8760  # p_out   # let's set all the values to 0
pv2.execute()
grid2.execute()
ur2.execute()
so2.execute()

results = pv2.Outputs.export()


In [21]:
# Sanity check 
dn = results['dn'][0:24]
dhi = results['df'][0:24]
alb = results['alb'][0:24]
poa = results['subarray1_poa_front'][0:24]
power = results['subarray1_dc_gross'][0:24]
pd.DataFrame(zip(dn, dhi, alb, poa, power), columns=['dn', 'dhi', 'alb', 'poa', 'power'])

,dn,dhi,alb,poa,power
0,0.0,0.0,0.8,0.000000,0.0
1,0.0,0.0,0.8,0.000000,0.0
2,0.0,0.0,0.8,0.000000,0.0
3,0.0,0.0,0.8,0.000000,0.0
4,0.0,0.0,0.8,0.000000,0.0
5,0.0,0.0,0.8,0.000000,0.0
6,0.0,0.0,0.8,0.000000,0.0
7,0.0,0.0,0.8,0.000000,0.0
8,0.0,16.0,0.8,13.363913,0.0
9,0.0,38.0,0.8,31.858243,0.0
